<a href="https://colab.research.google.com/github/elizabethavargas/cloud-hw1/blob/master/other-scripts/yelp_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Upload Yelp Restaurant Data to DynamoDB

In [1]:
# install packages
!pip install awscli
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9.1
    Uninstalling rsa-4.9.1:
      Successfully uninstalled rsa-4.9.1
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


### Open Saved Data

In [2]:
import pickle
with open('yelp_restaurants.pickle', 'rb') as f:
  yelp_restaurants = pickle.load(f)

In [3]:
yelp_restaurants[1]

{'id': 'mNeyjjcfrXdGf_mCyoolkw',
 'alias': 'lolas-new-york',
 'name': "Lola's",
 'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/mqY1TlTF7-zwNlR1hjDlJg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/lolas-new-york?adjust_creative=p6mhJwsttIzMzP5Ln3malA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=p6mhJwsttIzMzP5Ln3malA',
 'review_count': 80,
 'categories': [{'alias': 'newamerican', 'title': 'New American'}],
 'rating': 4.6,
 'coordinates': {'latitude': 40.74481688811193, 'longitude': -73.9878205},
 'transactions': [],
 'location': {'address1': '2 W 28th St',
  'address2': '',
  'address3': None,
  'city': 'New York',
  'zip_code': '10001',
  'country': 'US',
  'state': 'NY',
  'display_address': ['2 W 28th St', 'New York, NY 10001']},
 'phone': '',
 'display_phone': '',
 'distance': 2216.123384764452,
 'business_hours': [{'open': [{'is_overnight': False,
     'start': '1700',
     'end': '2100',
     'day': 0},
    {'is_overnight': False, 

In [12]:
r = yelp_restaurants[1]
item ={
    "businessId": r["id"],
    "name": r["name"],
    "address": r["location"]["address1"],
    "zipCode": r["location"]["zip_code"],
    "coordinates": {
            "latitude": r["coordinates"]["latitude"],
            "longitude": r["coordinates"]["longitude"]
        },
    "numberOfReviews": r["review_count"],
    "rating": r["rating"],
    "insertedAtTimestamp": datetime.utcnow().isoformat()
}
item

/tmp/ipython-input-2704250883.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "insertedAtTimestamp": datetime.utcnow().isoformat()


{'businessId': 'mNeyjjcfrXdGf_mCyoolkw',
 'name': "Lola's",
 'address': '2 W 28th St',
 'zipCode': '10001',
 'coordinates': {'latitude': 40.74481688811193, 'longitude': -73.9878205},
 'numberOfReviews': 80,
 'rating': 4.6,
 'insertedAtTimestamp': '2026-02-18T19:00:35.056256'}

### Insert Restaurants into DB

In [34]:
from google.colab import userdata

# load AWS key from secrets
AWS_KEY = userdata.get('AWS_KEY').strip()
AWS_SECRET = userdata.get('AWS_SECRET').strip()

In [37]:
import boto3
from datetime import datetime
from pytz import UTC
from decimal import Decimal

dynamodb = boto3.resource(
    "dynamodb",
    region_name="us-east-1",
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)

table = dynamodb.Table("yelp-restaurants")

for r in yelp_restaurants:
    item ={
      "businessId": r["id"],
      "name": r["name"],
      "address": r["location"]["address1"],
      "zipCode": r["location"]["zip_code"],
      "coordinates": {
              "latitude": Decimal(str(r["coordinates"]["latitude"])),
              "longitude": Decimal(str(r["coordinates"]["longitude"]))
          },
      "numberOfReviews": r["review_count"],
      "rating": Decimal(str(r['rating'])),
      "insertedAtTimestamp": datetime.now(UTC).isoformat()
    }

    table.put_item(Item=item)


### Check the Table

In [50]:
response = table.scan()
items = response['Items']

print("Number of items:", len(items))
print(items[:3])  # show first 3


Number of items: 1189
[{'businessId': 'IhSVn0TaX8xXb3wcQ-fgcA', 'numberOfReviews': Decimal('2922'), 'rating': Decimal('3.9'), 'insertedAtTimestamp': '2026-02-18T19:31:37.091241+00:00', 'address': '118A Eldridge St', 'zipCode': '10002', 'name': "Vanessa's Dumpling House", 'coordinates': {'latitude': Decimal('40.71839'), 'longitude': Decimal('-73.9917399')}}, {'businessId': 'ypqK8DWM8Bcs43YveSJnNw', 'numberOfReviews': Decimal('910'), 'rating': Decimal('4.4'), 'insertedAtTimestamp': '2026-02-18T19:31:22.798297+00:00', 'address': '28 Liberty St', 'zipCode': '10005', 'name': 'Manhatta', 'coordinates': {'latitude': Decimal('40.70800627689857'), 'longitude': Decimal('-74.00888226517013')}}, {'businessId': 'QPEzLkO35OTYijFhs8d7-w', 'numberOfReviews': Decimal('195'), 'rating': Decimal('4.4'), 'insertedAtTimestamp': '2026-02-18T19:31:21.597470+00:00', 'address': '16 E 46th St', 'zipCode': '10017', 'name': 'The Reading Room', 'coordinates': {'latitude': Decimal('40.7555'), 'longitude': Decimal('-